In [1]:
# Imports
import csv
import os
import pickle as pkl
import pandas as pd
import argparse
import cond_rnn

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
import kerastuner
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from tensorflow.keras.callbacks import TensorBoard

import tools.analysis as ta
from tools import keras as tk
import tools.preprocessing as tp

In [ ]:
 # GLOBALS   
TIME_SEQ = 225
TARGET = "multi_class"
BATCH_SIZE = 128
EPOCHS = 15
MAX_TRIALS = 500
TEST_SPLIT = 0.2
VAL_SPLIT = 0.1
RAND = 2021
TB_UPDATE_FREQ = 200
WEIGHTED_LOSS = False

# Paths
pwd = globals()['_dh'][0]

output_dir = os.path.abspath(os.path.join(pwd, "..", "output"))
data_dir = os.path.abspath(os.path.join(pwd, "..", "data", "data"))
tensorboard_dir = os.path.abspath(
    os.path.join(data_dir, "..", "model_checkpoints"))
pkl_dir = os.path.join(output_dir, "pkl")
stats_dir = os.path.join(output_dir, "analysis")

# Create analysis dir if it doesn't exist
os.makedirs(stats_dir, exist_ok=True)

stats_filename = TARGET + "_stats.csv"

In [ ]:
# Data load
with open(os.path.join(pkl_dir, TARGET + "_trimmed_seqs.pkl"), "rb") as f:
    inputs = pkl.load(f)

with open(os.path.join(pkl_dir, "all_ftrs_dict.pkl"), "rb") as f:
    vocab = pkl.load(f)

with open(os.path.join(pkl_dir, "feature_lookup.pkl"), "rb") as f:
    all_feats = pkl.load(f)

with open(os.path.join(pkl_dir, "demog_dict.pkl"), "rb") as f:
    demog_lookup = pkl.load(f)

# Determining number of vocab entries
N_VOCAB = len(vocab) + 1
N_DEMOG = len(demog_lookup) + 1
MAX_DEMOG = max(len(x) for _, x, _ in inputs)
N_CLASS = max(x for _, _, x in inputs) + 1


In [ ]:
# Model Metrics and callbacks
callbacks = [
    TensorBoard(
    log_dir=os.path.join(tensorboard_dir, "lstm_hp_tune_tb", ""),
    histogram_freq=1,
    profile_batch=0,
    write_graph=False,
    update_freq=TB_UPDATE_FREQ
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss",
                                min_delta=0,
                                patience=3,
                                restore_best_weights=True,
                                mode="min")
]

# Create some metrics
metrics = [
    keras.metrics.AUC(num_thresholds=int(1e5), name="ROC-AUC"),
    keras.metrics.AUC(num_thresholds=int(1e5), curve="PR", name="PR-AUC"),
    # NOTE: I think F1 Score is kind of wonky here, but I'll add it anyways
    # because that's pretty cool.
    tfa.metrics.F1Score(num_classes=N_CLASS, average="weighted")
]

# Define loss function
# NOTE: We were experimenting with focal loss at one point, maybe we can try that again at some point
# loss_fn = keras.losses.categorical_crossentropy if TARGET == "multi_class" else keras.losses.binary_crossentropy


In [ ]:
# TTV
# Splitting the data
train, test = train_test_split(
    range(len(inputs)),
    test_size=TEST_SPLIT,
    stratify=[labs for _, _, labs in inputs],
    random_state=RAND)

train, validation = train_test_split(
    train,
    test_size=VAL_SPLIT,
    stratify=[samp[2] for i, samp in enumerate(inputs) if i in train],
    random_state=RAND)

train_gen = tk.create_ragged_data_gen(
    [inputs[samp] for samp in train],
    max_demog=MAX_DEMOG,
    epochs=EPOCHS,
    multiclass=N_CLASS > 2,
    random_seed=RAND,
    batch_size=BATCH_SIZE)

validation_gen = tk.create_ragged_data_gen(
    [inputs[samp] for samp in validation],
    max_demog=MAX_DEMOG,
    epochs=EPOCHS,
    shuffle=False,
    multiclass=N_CLASS > 2,
    random_seed=RAND,
    batch_size=BATCH_SIZE)

# NOTE: don't shuffle test data
test_gen = tk.create_ragged_data_gen([inputs[samp] for samp in test],
                                        max_demog=MAX_DEMOG,
                                        epochs=1,
                                        multiclass=N_CLASS > 2,
                                        shuffle=False,
                                        random_seed=RAND,
                                        batch_size=BATCH_SIZE)

# %% Compute steps-per-epoch
# NOTE: Sometimes it can't determine this properly from tf.data
STEPS_PER_EPOCH = np.ceil(len(train) / BATCH_SIZE)
VALID_STEPS_PER_EPOCH = np.ceil(len(validation) / BATCH_SIZE)

In [ ]:
classes = np.unique([labs for _, _, labs in inputs]).tolist()

if WEIGHTED_LOSS:
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=classes,
        y=[labs for _, _, labs in inputs],
    )

    class_weights = dict(zip(classes, class_weights))

    print(class_weights)

## Generate Hypermodel

In [ ]:
hyper_model = tk.LSTMHyper(
    vocab_size = N_VOCAB,
    metrics = metrics,
    n_classes = N_CLASS,
    n_demog = N_DEMOG,
    n_demog_bags=MAX_DEMOG
)

tuner = kerastuner.tuners.BayesianOptimization(
    hyper_model,
    max_trials=MAX_TRIALS,
    objective="val_loss",
    project_name="lstm_hp_tune",
    # NOTE: This could be in output as well if we don't want to track/version it
    directory=tensorboard_dir,
)

## Search

In [ ]:
# Announce the search space
tuner.search_space_summary()

In [ ]:
tuner.search(train_gen,
             validation_data=validation_gen,
             epochs=EPOCHS,
             steps_per_epoch=STEPS_PER_EPOCH,
             validation_steps=VALID_STEPS_PER_EPOCH,
             callbacks=callbacks
)

## Pull Best Model

In [ ]:
tuner.results_summary()

In [ ]:
# Pull the best model
best_hp = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hp)

best_model.summary()

In [ ]:
best_model.save(os.path.join(tensorboard_dir, "lstm_hp_tune", "best"))